In [1]:
import sqlite3, os, sys
import pandas as pd
import tensorflow as tf
import numpy as np
from tqdm.notebook import tqdm

In [6]:
# dfjs=pd.read_csv('~/../jakobsch/predictions/jakob_predictions_0106_2.csv').sort_values(by='event_no')
# dfjs=pd.read_csv('~/../jakobsch/predictions/jakob_predictions_0806_new.csv').sort_values(by='event_no')

np.percentile(abs(dfjs['zenith']-dfjs['zenith_pred'])*180/np.pi,50)

17.247221909961098

In [6]:
transform_path= "../db_files/muongun/transformers.pkl"
db_path="../db_files/muongun/rasmus_classification_muon_3neutrino_3mio.db"
features=   ["event_no", "dom_x", "dom_y", "dom_z", "time", "charge_log10", "SRTInIcePulses", "string", "dom", "lc", "pulse_width"]
set_path= "../db_files/muongun/sets.pkl"
targets=    ["event_no", "energy_log10", "zenith","azimuth"]
conn = sqlite3.connect(db_path)
def get_event_no():
        print('Reading sets')
        sets = pd.read_pickle(set_path)
        train_events = sets['train']
        test_events = sets['test']
        return train_events.to_numpy(), test_events.to_numpy()
_, test_events=get_event_no()
df_feat=pd.read_sql(f"select * from features where event_no in {tuple(test_events)} limit 100", conn)
df_feat.head()

Reading sets


,row,event_no,string,dom,pmt,dom_x,dom_y,dom_z,pmt_x,pmt_y,pmt_z,pmt_area,pmt_type,time,charge_log10,lc,pulse_width,SplitInIcePulses,SRTInIcePulses
0,91242041,138674340,1,13,0,-1.206695,-2.036696,1.006918,0.0,0.0,-1.0,0.0444,20,-0.902557,-0.088028,0,8,1,0
1,91242042,138674340,59,43,0,1.879185,1.226374,0.000503,0.0,0.0,-1.0,0.0444,20,-1.099296,0.378965,0,8,1,0
2,91242043,138674340,63,30,0,-0.438924,1.306133,0.428594,0.0,0.0,-1.0,0.0444,20,0.424602,0.791982,0,8,1,0
3,91242044,138674340,66,29,0,1.061076,1.552111,0.493253,0.0,0.0,-1.0,0.0444,20,-0.125602,1.160970,0,8,1,0
4,91242045,138674340,66,29,0,1.061076,1.552111,0.493253,0.0,0.0,-1.0,0.0444,20,-0.114116,-1.656923,0,8,1,0


In [7]:
feature_call = ", ".join(features)
df_feat=pd.read_sql(f"select {feature_call} from features where event_no in {tuple(test_events)}", conn)
cols=df_feat.columns

In [8]:
import pickle
transformers = pickle.load(open(transform_path, 'rb'))
trans_y      = transformers['features']
trans_keys=trans_y.keys()
trans_keys

/groups/hep/chri862z/anaconda3/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator RobustScaler from version 0.19.1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


dict_keys(['dom_x', 'dom_y', 'dom_z', 'time', 'charge_log10'])

In [9]:
for col in list(trans_keys)[:-1]:
    df_feat[col] = trans_y[col].inverse_transform(np.array(df_feat[col]).reshape(1, -1)).T

In [10]:
df_feat=df_feat.sort_values(by='event_no')

In [11]:
event_nos, counts = np.unique(df_feat['event_no'], return_counts=1)

In [ ]:
# df_clean=df_feat['rqe']

In [12]:
x_not_split=np.array(df_feat.drop('event_no', axis=1))
xs = np.split(x_not_split, np.cumsum(counts)[:-1])

In [13]:
n_strings=np.zeros(len(xs))
n_layers=np.zeros(len(xs))
cq=np.zeros((len(xs), 3))
for i in tqdm(range(len(xs))):
    _ , nlayers = np.unique(xs[i][:,2], return_counts=True)
    # xy=[tuple(i) for i in xs[i][:,:2]]
    _ , nstring = np.unique(xs[i][:,:2], return_counts=True, axis=0)
    cqe=xs[i][:,5]/np.sum(xs[i][:,5])
    cq[i]=np.sum(np.array([cqe,cqe,cqe]).T*xs[i][:,:3], axis=0)
    n_layers[i]=len(nlayers)
    n_strings[i]=len(nstring)

In [14]:
n_doms=counts

In [15]:
df_stats=pd.DataFrame(event_nos, columns=['event_no'])

In [16]:
df_stats['n_strings']=n_strings
df_stats['n_doms']=n_doms
for i, x in enumerate(['x', 'y', 'z']):
    df_stats['cq_'+x]=cq[:,i]

In [17]:
df_stats.describe() 

,event_no,n_strings,n_doms,cq_x,cq_y,cq_z
count,3.000000e+05,300000.000000,300000.000000,300000.000000,300000.000000,300000.000000
mean,1.396108e+08,30.981220,69.138490,6.712554,-5.219511,-60.572678
std,5.407247e+05,6.532807,52.943605,259.947137,242.956764,250.754102
min,1.386743e+08,8.000000,14.000000,-570.900024,-521.080017,-501.919998
25%,1.391421e+08,26.000000,47.000000,-195.954180,-181.594573,-307.806312
50%,1.396108e+08,30.000000,59.000000,16.543605,-16.706618,-26.204460
75%,1.400793e+08,35.000000,76.000000,197.117273,176.456249,158.723654
max,1.405476e+08,74.000000,3535.000000,576.369995,509.500000,497.626921


In [18]:
df_feat_srt=df_feat[df_feat['SRTInIcePulses']==1]

In [20]:
event_nos, counts = np.unique(df_feat_srt['event_no'], return_counts=1)
x_not_split=np.array(df_feat_srt.drop('event_no', axis=1))
xs = np.split(x_not_split, np.cumsum(counts)[:-1])
n_strings=np.zeros(len(xs))
n_layers=np.zeros(len(xs))
cq=np.zeros((len(xs), 3))
for i in tqdm(range(len(xs))):
    _ , nlayers = np.unique(xs[i][:,2], return_counts=True)
    # xy=[tuple(i) for i in xs[i][:,:2]]
    _ , nstring = np.unique(xs[i][:,:2], return_counts=True, axis=0)
    cqe=xs[i][:,5]/np.sum(xs[i][:,5])
    cq[i]=np.sum(np.array([cqe,cqe,cqe]).T*xs[i][:,:3], axis=0)
    n_layers[i]=len(nlayers)
    n_strings[i]=len(nstring)
n_doms=counts
df_stats['n_strings_SRT']=n_strings
df_stats['n_doms_SRT']=n_doms
for i, x in enumerate(['x', 'y', 'z']):
    df_stats['cq_'+x+'_SRT']=cq[:,i]

In [21]:
df_stats

,event_no,n_strings,n_doms,cq_x,cq_y,cq_z,n_strings_SRT,n_doms_SRT,cq_x_SRT,cq_y_SRT,cq_z_SRT
0,138674340,27.0,45,-193.186920,458.876916,-436.897693,4.0,13,-193.186920,458.876916,-436.897693
1,138674343,29.0,64,119.397646,256.359412,202.360588,4.0,17,119.397646,256.359412,202.360588
2,138674348,39.0,60,371.198826,219.410585,-340.860587,7.0,17,371.198826,219.410585,-340.860587
3,138674358,28.0,82,288.488726,433.885323,145.040213,6.0,47,288.488726,433.885323,145.040213
4,138674364,32.0,58,-47.756428,-409.500706,50.514286,5.0,14,-47.756428,-409.500706,50.514286
...,...,...,...,...,...,...,...,...,...,...,...
299995,140547624,41.0,157,61.821713,-50.189819,-275.323783,17.0,111,61.821713,-50.189819,-275.323783
299996,140547629,26.0,58,-144.056129,-121.436773,-93.644838,9.0,31,-144.056129,-121.436773,-93.644838
299997,140547636,33.0,77,-157.693588,64.871281,-61.076667,9.0,39,-157.693588,64.871281,-61.076667
299998,140547642,36.0,53,-201.143078,-284.856159,-59.483844,7.0,13,-201.143078,-284.856159,-59.483844


In [22]:
df_feat_nsrt=df_feat[df_feat['SRTInIcePulses']==0]
df_feat_nsrt.describe()

,event_no,dom_x,dom_y,dom_z,time,charge_log10,SRTInIcePulses,string,dom,lc,pulse_width
count,1.148379e+07,1.148379e+07,1.148379e+07,1.148379e+07,1.148379e+07,1.148379e+07,11483792.0,1.148379e+07,1.148379e+07,1.148379e+07,1.148379e+07
mean,1.396108e+08,9.771001e-01,-2.851579e+00,-2.156857e+01,1.191810e+04,-1.301159e-01,0.0,4.464796e+01,3.000617e+01,2.099655e-02,7.915011e+00
std,5.407778e+05,2.798908e+02,2.656833e+02,3.011649e+02,3.543241e+03,7.765054e-01,0.0,2.529617e+01,1.754814e+01,1.433726e-01,7.665295e-01
min,1.386743e+08,-5.709000e+02,-5.210800e+02,-5.128200e+02,5.713000e+03,-5.720069e+00,0.0,1.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00
25%,1.391420e+08,-2.240900e+02,-1.945000e+02,-2.935700e+02,9.004000e+03,-4.991211e-01,0.0,2.300000e+01,1.500000e+01,0.000000e+00,8.000000e+00
50%,1.396110e+08,1.187000e+01,3.490000e+00,-2.697000e+01,1.187000e+04,0.000000e+00,0.0,4.500000e+01,3.000000e+01,0.000000e+00,8.000000e+00
75%,1.400792e+08,2.104700e+02,2.029800e+02,2.407200e+02,1.464100e+04,3.789646e-01,0.0,6.700000e+01,4.500000e+01,0.000000e+00,8.000000e+00
max,1.405476e+08,5.763700e+02,5.095000e+02,5.245600e+02,3.348600e+04,5.919837e+00,0.0,8.600000e+01,6.000000e+01,1.000000e+00,8.000000e+00


In [23]:
df_feat_srt.describe()

,event_no,dom_x,dom_y,dom_z,time,charge_log10,SRTInIcePulses,string,dom,lc,pulse_width
count,9.257755e+06,9.257755e+06,9.257755e+06,9.257755e+06,9.257755e+06,9.257755e+06,9257755.0,9.257755e+06,9.257755e+06,9.257755e+06,9.257755e+06
mean,1.396111e+08,1.666149e+01,-1.326632e+01,-1.179158e+02,1.122363e+04,-1.856653e-01,1.0,4.975738e+01,3.380178e+01,7.501382e-01,3.129374e+00
std,5.412690e+05,2.381423e+02,2.243531e+02,2.720490e+02,1.276934e+03,9.376247e-01,0.0,2.603076e+01,1.660687e+01,4.329329e-01,3.220280e+00
min,1.386743e+08,-5.709000e+02,-5.210800e+02,-5.128200e+02,5.720000e+03,-5.720069e+00,1.0,1.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00
25%,1.391409e+08,-1.328000e+02,-1.312500e+02,-3.532400e+02,1.024200e+04,-6.210354e-01,1.0,2.700000e+01,2.000000e+01,1.000000e+00,1.000000e+00
50%,1.396110e+08,4.160000e+01,-3.488000e+01,-1.970200e+02,1.096000e+04,0.000000e+00,1.0,4.900000e+01,3.400000e+01,1.000000e+00,1.000000e+00
75%,1.400794e+08,1.320300e+02,1.272000e+02,1.362200e+02,1.190000e+04,3.789646e-01,1.0,7.800000e+01,4.900000e+01,1.000000e+00,8.000000e+00
max,1.405476e+08,5.763700e+02,5.095000e+02,5.245600e+02,3.054100e+04,9.752296e+00,1.0,8.600000e+01,6.000000e+01,1.000000e+00,8.000000e+00


In [24]:
df_stats.to_csv('../db_files/muongun_stats.csv')

In [58]:
xy=[tuple(i) for i in xs[k][:,:2]]
vals  = np.unique(xy)
vals, xy

(array([-131.25      , -105.51999664,  -79.5       ,  -66.59999847,
         -60.47000122,  -10.97000027,   -9.68000031,    6.71999979,
          27.09000015,   57.20000076,   72.37000275,  106.94000244,
         113.19000244,  124.97000122]),
 [(-10.970000267028809, 6.71999979019165),
  (-10.970000267028809, 6.71999979019165),
  (-9.680000305175781, -79.5),
  (57.20000076293945, -105.5199966430664),
  (113.19000244140625, -60.470001220703125),
  (113.19000244140625, -60.470001220703125),
  (113.19000244140625, -60.470001220703125),
  (113.19000244140625, -60.470001220703125),
  (113.19000244140625, -60.470001220703125),
  (106.94000244140625, 27.09000015258789),
  (72.37000274658203, -66.5999984741211),
  (72.37000274658203, -66.5999984741211),
  (72.37000274658203, -66.5999984741211),
  (124.97000122070312, -131.25),
  (124.97000122070312, -131.25),
  (113.19000244140625, -60.470001220703125)])

In [33]:
event_nocall

NameError: name 'event_nocall' is not defined

In [ ]:
stop_id.fetchall()

In [ ]:
start, stop = 0, 100000
# start_id  = conn.execute(f"select distinct event_no from features where pid = 13 limit ")
event_nos = pd.read_sql(f"select distinct event_no from features limit {stop} offset {start}", conn)

In [ ]:
event_nos

In [ ]:
start, stop = 0, 100000
select_rows = ["dom_x", "dom_y", "dom_z", "time", "charge_log10"]

start_id = conn.execute(f"select distinct event_no from features limit 1 offset {start}").fetchall()[0][0]
stop_id  = conn.execute(f"select distinct event_no from features limit 1 offset {stop}").fetchall()[0][0]

In [ ]:
features = ["dom_x", "dom_y", "dom_z", "time", "charge_log10"]
targets  = ["energy_log10", "position_x", "position_y", "position_z", "direction_x", "direction_y", "direction_z"]
feature_call = ", ".join(features)
target_call  = ", ".join(targets)
df_event = pd.read_sql(f"select event_no       from features where event_no >= {start_id} and event_no < {stop_id}", conn)
df_feat  = pd.read_sql(f"select {feature_call} from features where event_no >= {start_id} and event_no < {stop_id}", conn)
df_targ  = pd.read_sql(f"select {target_call } from truth    where event_no >= {start_id} and event_no < {stop_id}", conn)

In [ ]:
idx_list    = np.array(df_event)
x_not_split = np.array(df_feat)

In [ ]:
np.shape(idx_list)

In [ ]:
ids, idx, counts = np.unique(idx_list.flatten(), return_index = True, return_counts = True) 

In [ ]:
np.sum(counts)

In [ ]:
x_not_split.shape

In [ ]:
xs = np.split(x_not_split, np.cumsum(counts))

In [ ]:
ys = np.array(df_targ)

In [ ]:
np.shape(xs)

In [ ]:
for x, y in zip(xs, ys):
    print(x, y)
    break

In [ ]:
from sklearn.neighbors import kneighbors_graph as knn
from spektral.data import Graph
neighbors=6

In [ ]:
print("Generating adjacency matrices")
graph_list = []
for x, y in tqdm(zip(xs, ys), total = len(xs)):
    a = knn(x[:, :3], neighbors)



    graph_list.append(Graph(x = x, a = a, y = y))

graph_list = np.array(graph_list, dtype = object)

In [ ]:
s = [x.shape[0] for x in xs]

In [ ]:
xs = np.array(xs, dtype = object)[np.argsort(s)]

In [ ]:
size, idx = np.unique(np.sort(s), return_index = True)

In [ ]:
preprocess(list(xs[4: 9]))

In [ ]:
np.sort(s)

In [ ]:
idx

In [ ]:
import pickle
transformers = pickle.load(open('../db_files/muongun/transformers.pkl', 'rb'))
trans_x      = transformers['features']
trans_y      = transformers['truth']

In [15]:
diff={'hyper_params':{'hidden_states': [128,64,32],
     'dropout': [0,0.05],
     'conv_layers': [1,2,3],
     'decode_layers': [2,4]},
     'run_params':{'batch_size':[512,1024,2048]}}

In [23]:
diff={'hidden_states': [128,64,32],
     'dropout': [0,0.05],
     'conv_layers': [1,2,3],
     'decode_layers': [2,4],
     'batch_size':[512,1024,2048]}

In [42]:
def perms(diffs):
    from itertools import product
    keys=list(diffs.keys())
    val=list(diffs.values())
    for i, s in enumerate(val):
        if i==0:
            a=val[0]
        else:
            a=product(a, val[i])
    bs=[]
    for b in a:
        bs.append(b)
    output=[]
    def removeNestings(l):
        for i in l:
            if type(i) == tuple:
                removeNestings(i)
            else:
                output.append(i)
    removeNestings(bs)
    perms=np.array(output)
#     perms=[int(num) if float(num).is_integer() else num for num in perms]
    return perms.reshape(-1, len(keys))

In [47]:
keys=list(diff.keys())
for j, key in enumerate(keys):
    if key in base['run_params']:
        base['run_params'][key]=p[0][j]
    elif key in base['hyper_params']:
        typ=type(base['hyper_params'][key])
        base['hyper_params'][key]=typ(p[0][j])
    elif key in base['data_params']:
        base['data_params'][key]=p[0][j]

<class 'int'>
<class 'float'>
<class 'int'>
<class 'int'>


In [44]:
p=perms(diff)

In [38]:
p

array([[1.280e+02, 0.000e+00, 1.000e+00, 2.000e+00, 5.120e+02],
       [1.280e+02, 0.000e+00, 1.000e+00, 2.000e+00, 1.024e+03],
       [1.280e+02, 0.000e+00, 1.000e+00, 2.000e+00, 2.048e+03],
       [1.280e+02, 0.000e+00, 1.000e+00, 4.000e+00, 5.120e+02],
       [1.280e+02, 0.000e+00, 1.000e+00, 4.000e+00, 1.024e+03],
       [1.280e+02, 0.000e+00, 1.000e+00, 4.000e+00, 2.048e+03],
       [1.280e+02, 0.000e+00, 2.000e+00, 2.000e+00, 5.120e+02],
       [1.280e+02, 0.000e+00, 2.000e+00, 2.000e+00, 1.024e+03],
       [1.280e+02, 0.000e+00, 2.000e+00, 2.000e+00, 2.048e+03],
       [1.280e+02, 0.000e+00, 2.000e+00, 4.000e+00, 5.120e+02],
       [1.280e+02, 0.000e+00, 2.000e+00, 4.000e+00, 1.024e+03],
       [1.280e+02, 0.000e+00, 2.000e+00, 4.000e+00, 2.048e+03],
       [1.280e+02, 0.000e+00, 3.000e+00, 2.000e+00, 5.120e+02],
       [1.280e+02, 0.000e+00, 3.000e+00, 2.000e+00, 1.024e+03],
       [1.280e+02, 0.000e+00, 3.000e+00, 2.000e+00, 2.048e+03],
       [1.280e+02, 0.000e+00, 3.000e+00,

In [31]:
print([int(num) if float(num).is_integer() else num for num in p])

TypeError: only size-1 arrays can be converted to Python scalars

In [46]:
true, false=1,0
base={
    "experiment": "IceCube",
    "group":  "compare", 

    "model_name":  "Sage",
    "angle_unitvec": "angle",
    "wandblog": true,

    "run_params":{
    "epochs":     20,
    "batch_size": 512,
    "early_stop": true,
    "zeniazi_metric":true,
    "patience":   5,
    "lr_schedule": "classic",
    "learning_rate":   1e-3,
    "warm_up": 3,
    "lr_decay": 0.9,
    "val_epoch":   10,
    "loss_func": "abs_vonMises2D_angle",
    "metrics": "energy_angle_zeniazi",
    "performance_plot": "performance_vM2D"
},

    "hyper_params": {
        "n_out":           3,
        "n_sigs":          2,
        "hidden_states":   64,
        "glob": true,
        "conv_layers":   2,
        "conv_activation": "relu",
        "decode_layers":   3,
        "decode_activation": 1,
        "dropout":         0.05,
        "regularization": "l2", 
        "batch_norm":      true,
        "forward":  true,
        "edgeconv": 0,
        "edgenorm":0
    },

   "data_params":{ 
    "n_data": 2e4,
    "graph_construction":       "classic",
    "muon":             true,
    "n_neighbors":       6,
    "data_split":      [0.8, 0.1, 0.1],
    "restart":   false,
    "transform_path": "../db_files/muongun/transformers.pkl",
    "db_path": "../db_files/muongun/rasmus_classification_muon_3neutrino_3mio.db",
    "features":   ["dom_x", "dom_y", "dom_z", "time", "charge_log10", "SRTInIcePulses", "lc"],
    "targets":    ["energy_log10", "zenith","azimuth"],
    "skip": 0,
    "SRT": 0,
    "database": "MuonGun"
}
}

In [ ]:
l=bs
outp
  
# Driver code
print ('The original list: ', l)
removeNestings(l)
print ('The list after removing nesting: ', output)

In [ ]:
perms=np.array(output)
perms.reshape(-1, len(keys))

In [ ]:
bs=np.array(bs)

In [ ]:
bs.reshape(len(keys), len(bs))